# He I Diagnostics Using FIRS Data

## 3. Parallel inversion

Shuo Wang

Dept. of Astronomy, NMSU

DKIST Ambassador

### Load Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import readsav
from skimage.transform import downscale_local_mean
import hazel
import h5py

In [ ]:
le,r = 100,172 
s4 = np.load('clean.npy')[:,:,le:r] # spatial resolution dy=0.15,dx=0.3 arcsec
print(s4.shape)

In [ ]:
wva = readsav('wva.sav')['wva'][le:r]
nw = wva.shape[0]
print(f'{wva[0]:.3f}',f'{wva[-1]:.3f}',nw)

### Resize

In [ ]:
img = downscale_local_mean(s4[0,:,10], (6, 3)) # dx=dy=0.9 arcsec
nimg = img.shape
print(nimg)
sd9 = np.empty([4,nimg[0], nw,nimg[1]])
for i in range(4):
    for j in range(nw):
        sd9[i,:,j,:] = downscale_local_mean(s4[i,:,j], (6, 3)) # 0.9 arcsec
vmin,vmax = np.percentile(sd9[0,:,10],5),np.percentile(sd9[0,:,10],95)
plt.figure(figsize=(12,8))
plt.subplot(121)
plt.imshow(s4[0,:,10],origin='lower',vmin=vmin,vmax=vmax)
plt.subplot(122)
plt.imshow(sd9[0,:,10],origin='lower',vmin=vmin,vmax=vmax)

 ### Select Region Of Interest

In [ ]:
y1,x1 = 197,244
y1d9,x1d9 = y1//6,x1//3
ny = nx = 2
sd9r = sd9[:,y1d9:y1d9+ny,:,x1d9:x1d9+nx]

plt.figure(figsize=(12,8))
plt.plot(wva,s4[0,y1,:,x1]/np.max(s4[0,y1,:,x1]),label = 'px1')
plt.plot(wva,sd9r[0,0,:,0]/np.max(sd9r[0,0,:,0]),label = 'px1d9')
plt.axvline(x=10830.3,color='C3')
plt.legend()

### Prepare Input Files for HAZEL

In [ ]:
stokes = np.zeros((ny*nx,nw,4))
idx=0
for yi in range(ny):
    for xi in range(nx):
        stokes[idx] = sd9r[:,yi,:,xi].T
        stokes[idx,:,0] /=np.max(stokes[idx,:,0])
        idx += 1
sigma = np.array([[[1e-2,5e-4,5e-4,5e-4],]*nw,]*ny*nx, dtype=np.float64) # noise IQUV
los = np.array([[0, 0, 90],]*ny*nx, dtype=np.float64)
boundary = np.zeros((ny*nx,nw,4), dtype=np.float64)
boundary[:,:,0] = 1.0

f = h5py.File('in.h5', 'w')
db_stokes = f.create_dataset('stokes', stokes.shape, dtype=np.float64)
db_sigma = f.create_dataset('sigma', sigma.shape, dtype=np.float64)
db_los = f.create_dataset('LOS', los.shape, dtype=np.float64)
db_boundary = f.create_dataset('boundary', boundary.shape, dtype=np.float64)
db_stokes[:] = stokes
db_sigma[:] = sigma
db_los[:] = los
db_boundary[:] = boundary
f.close()

### Run HAZEL Inversion

In [ ]:
!mpiexec -n 5 python inve.py

### Results

In [ ]:
fo = h5py.File('output.h5', 'r')
ch1 = fo['ch1']
arr = np.array(['deltav','tau','v'])
for i in arr:
    print(i,': ',f'{ch1[i][0,0,0]:.2f}')

iq=['I','Q','U','V']
plt.figure(figsize = (12,8))
for i in range(4):
    plt.subplot(221+i)
    for j in [0,3]:
        plt.plot(wva, stokes[j,:,i],'.',label='observation'+str(j))
        plt.plot(wva, fo['spec1']['stokes'][j,0,i],label='inversion'+str(j))
    plt.xlabel('Wavelength [$\AA$]')
    plt.ylabel(iq[i])
plt.legend()
fo.close()